In [1]:
# ====== Import the required modules 
from geopy.distance import geodesic
import geopandas as gpd
import pandas as pd

In [2]:
# ====== Set pandas options for visibility and verification
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# ====== Read the fires data from the NIFC API via ArcGis
fire_perimeters = gpd.read_file('https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/WFIGS_Interagency_Perimeters_Current/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')
display(fire_perimeters.head(5))

,OBJECTID,poly_SourceOID,poly_IncidentName,poly_FeatureCategory,poly_MapMethod,poly_GISAcres,poly_DeleteThis,poly_FeatureAccess,poly_FeatureStatus,poly_IsVisible,poly_CreateDate,poly_DateCurrent,poly_PolygonDateTime,poly_IRWINID,poly_FORID,poly_Acres_AutoCalc,poly_SourceGlobalID,poly_Source,attr_SourceOID,attr_ABCDMisc,attr_ADSPermissionState,attr_CalculatedAcres,attr_ContainmentDateTime,attr_ControlDateTime,attr_CreatedBySystem,attr_IncidentSize,attr_DiscoveryAcres,attr_DispatchCenterID,attr_EstimatedCostToDate,attr_FinalAcres,attr_FFReportApprovedByTitle,attr_FFReportApprovedByUnit,attr_FFReportApprovedDate,attr_FireBehaviorGeneral,attr_FireBehaviorGeneral1,attr_FireBehaviorGeneral2,attr_FireBehaviorGeneral3,attr_FireCause,attr_FireCauseGeneral,attr_FireCauseSpecific,attr_FireCode,attr_FireDepartmentID,attr_FireDiscoveryDateTime,attr_FireMgmtComplexity,attr_FireOutDateTime,attr_FireStrategyConfinePercent,attr_FireStrategyFullSuppPrcnt,attr_FireStrategyMonitorPercent,attr_FireStrategyPointZonePrcnt,attr_FSJobCode,attr_FSOverrideCode,attr_GACC,attr_ICS209ReportDateTime,attr_ICS209RptForTimePeriodFrom,attr_ICS209RptForTimePeriodTo,attr_ICS209ReportStatus,attr_IncidentManagementOrg,attr_IncidentName,attr_IncidentShortDescription,attr_IncidentTypeCategory,attr_IncidentTypeKind,attr_InitialLatitude,attr_InitialLongitude,attr_InitialResponseAcres,attr_InitialResponseDateTime,attr_IrwinID,attr_IsFireCauseInvestigated,attr_IsFireCodeRequested,attr_IsFSAssisted,attr_IsMultiJurisdictional,attr_IsReimbursable,attr_IsTrespass,attr_IsUnifiedCommand,attr_LocalIncidentIdentifier,attr_ModifiedBySystem,attr_PercentContained,attr_PercentPerimToBeContained,attr_POOCity,attr_POOCounty,attr_POODispatchCenterID,attr_POOFips,attr_POOJurisdictionalAgency,attr_POOJurisdictionalUnit,attr_POOJurisdictUnitParentUnit,attr_POOLandownerCategory,attr_POOLandownerKind,attr_POOLegalDescPrincipalMerid,attr_POOLegalDescQtr,attr_POOLegalDescQtrQtr,attr_POOLegalDescRange,attr_POOLegalDescSection,attr_POOLegalDescTownship,attr_POOPredictiveServiceAreaID,attr_POOProtectingAgency,attr_POOProtectingUnit,attr_POOState,attr_PredominantFuelGroup,attr_PredominantFuelModel,attr_PrimaryFuelModel,attr_SecondaryFuelModel,attr_TotalIncidentPersonnel,attr_UniqueFireIdentifier,attr_FORID,attr_WFDSSDecisionStatus,attr_EstimatedFinalCost,attr_OrganizationalAssessment,attr_StratDecisionPublishDate,attr_CreatedOnDateTime_dt,attr_ModifiedOnDateTime_dt,attr_Source,attr_ActiveFireCandidate,attr_IsCpxChild,attr_CpxName,attr_CpxID,attr_SourceGlobalID,GlobalID,Shape__Area,Shape__Length,attr_IncidentComplexityLevel,geometry
0,37190,2217,Lebar,Wildfire Daily Fire Perimeter,Mixed Methods,NaN,No,Public,Approved,Yes,1740422477000,1740759310000,NaN,{DB5C769E-79D3-48CB-B9DD-80E56E3063EE},None,512.723102,{F21880FE-EACC-48C9-9909-087C84CACB68},2025 NIFS,33055974,None,DEFAULT,NaN,None,None,wildcade,512.00,30.0,WYCPC,2000.0,None,None,None,None,Active,None,Wind Driven Runs,Running,Human,None,None,SFV7,None,1740335400000,None,None,0.0,100.0,0.0,0.0,None,None,RMCC,1.740434e+12,1.740341e+12,1.740424e+12,F,Type 4 IC,Lebar,10 miles north of Douglas,WF,FI,42.919821,-105.267220,None,NaN,{DB5C769E-79D3-48CB-B9DD-80E56E3063EE},NaN,0,1.0,None,None,None,0.0,250017,ics209,100.0,100.0,Douglas,Converse,WYCPC,56009,None,None,None,Private,Private,None,None,None,None,None,None,RM04,C&L,WYCOX,US-WY,None,None,Short Grass (1 foot),Tall Grass (2.5 feet),32.0,2025-WYCOX-250017,None,No Decision,NaN,None,NaN,1740365174000,1740433893000,IRWIN,1,0,None,None,{F0353CBC-81CE-4657-819B-7A8CE3AFA40E},741e8949-172d-463f-98d7-83670de6d935,0.000229,0.152801,None,"POLYGON ((-105.26445 42.89318, -105.26457 42.8..."
1,37161,2428,White face,Wildfire Daily Fire Perimeter,Mixed Methods,43.500000,No,Public,Approved,Yes,1740602967000,1740604817000,1.740603e+12,{9B77A8AE-60DC-436B-9FD3-58F259348034},None,43.462441,{B095F25C-D728-4AFA-B5A7-CCA5416669CA},2025 NIFS,33056261,None,DEFAULT,43.44,None,None,INFORM_Inspector,43.44,0.1,SDGPC,NaN,None,

In [4]:
# ====== Function to create distances for many values in a dataframe
def calculate_point_distances(gdf):
    # Keep only the required columns and drop rows with NaN in coordinates
    base_df = gdf[['OBJECTID', 'attr_InitialLatitude', 'attr_InitialLongitude']].dropna(
        subset=['attr_InitialLatitude', 'attr_InitialLongitude']
    )
    
    # Create an empty list to store results
    results = []
    
    # Iterate through each point as the reference point
    for idx1, row1 in base_df.iterrows():
        point1 = (row1['attr_InitialLatitude'], row1['attr_InitialLongitude'])
        
        # Compare with all other points
        for idx2, row2 in base_df.iterrows():
            if idx1 != idx2:  # Skip comparing point with itself
                point2 = (row2['attr_InitialLatitude'], row2['attr_InitialLongitude'])
                
                # Calculate distance using geodesic
                distance = geodesic(point1, point2)
                
                # Create record with all required information
                result = {
                    'OBJECTID': row1['OBJECTID'],
                    'attr_InitialLatitude': row1['attr_InitialLatitude'],
                    'attr_InitialLongitude': row1['attr_InitialLongitude'],
                    'neighborLatitude': row2['attr_InitialLatitude'],
                    'neighborLongitude': row2['attr_InitialLongitude'],
                    'distance_km': distance.kilometers,
                    'distance_miles': distance.miles
                }
                results.append(result)
    
    # Convert results to DataFrame
    result_df = pd.DataFrame(results)
    
    return result_df

In [5]:
# ====== Run the function to determine the distances for each location to all other locations
result = calculate_point_distances(fire_perimeters)
display(result)

,OBJECTID,attr_InitialLatitude,attr_InitialLongitude,neighborLatitude,neighborLongitude,distance_km,distance_miles
0,37190.0,42.919821,-105.267220,30.179200,-84.996300,2290.094496,1422.998747
1,37190.0,42.919821,-105.267220,26.158153,-80.890974,2890.173998,1795.870863
2,37190.0,42.919821,-105.267220,36.842220,-91.083183,1385.697533,861.032528
3,37190.0,42.919821,-105.267220,36.738889,-96.194167,1035.338669,643.329623
4,37190.0,42.919821,-105.267220,43.400694,-103.511682,152.439262,94.721366
5,37190.0,42.919821,-105.267220,30.202883,-84.425450,2328.263043,1446.715583
6,37190.0,42.919821,-105.267220,35.412100,-80.071200,2320.616565,1441.964281
7,37190.0,42.919821,-105.267220,36.848582,-91.182146,1377.952717,856.220122
8,37190.0,42.919821,-105.267220,33.804617,-78.859600,2506.279586,1557.329935
9,37190.0,42.919821,-105.267220,46.575088,-94.654813,932.393488,579.362453


In [6]:
# ====== For individual points, use this function in a manual context
distance_miles = geodesic((42.919821, -105.267220), (43.400694, -103.511682)).miles
display(distance_miles)

94.7213660136532